In [4]:
# Quick hack to put us in the root of the repository/pipeline directory
import os
if os.path.exists("01.cli_demonstration.ipynb"):
    os.chdir("..")
print("Running in {}".format(os.getcwd()))

Running in /data/hpcdata/users/jambyr/icenet/notebook-test


# IceNet Library Usage

## Context

### Purpose
The IceNet library provides the ability to download, process, train and predict from end to end via a set of command-line interfaces.

Using this notebook we can understand how to programmatically undertake various activities using the IceNet library, allowing for significant customisation of the end to end deep learning pipeline for research and operational use.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentraion forecasts.

### Highlights
The key features of an end to end run are: 
* Setup: _this was concerned with setting up the conda environment, which remains the same as in 01.cli_demonstration_
* [Download](#Download) 
* [Process](#Process)
* [Train](#Train)
* [Predict](#Predict)

_This follows the same structure as the CLI demonstration notebook so that it's easy to follow step-by-step..._

### Contributions
#### Notebook
James Byrne (author)

__Please raise issues [in this repository](https://github.com/antarctica/IceNet-Pipeline) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

## Introduction

Once installed the API can be utilised as easily as the CLI commands from a shell, via any Python interpreter. As usual ensure that you're operating within the conda environment you installed the library into.

### A tip on CLI - API usage

All of the `icenet_*` CLI commands behind the scenes implement API activities. By inspecting the [`setup.py` ](https://github.com/JimCircadian/icenet2/blob/main/setup.py#L32) entry points you can locate the module and thus the code used by these. 

In most cases the CLI imposes various assumptions about what to do without exposing, necessarily, all available options to change the behaviour of the library. This is primarily as the CLI entry points are still under development to open up the options, so these CLI operations are for introductory use and API usage is recommended for advanced use cases and pipeline integrations.

### What we'll cover

For the sake of illustration this notebook will display and execute the equivalent API code, equivalent to [the first notebook of this collection](01.cli_demonstration.ipynb) as well as some updates that incorporate the visualisations from [the second notebook describing the data](02.data_and_forecasts.ipynb). However, for the sake of extending our dataset, we'll work towards extending our original downloads from covering *2019-12-28 through 2020-4-30* to cover 2020 in totality, as well as creating a more complex selection of dates for our dataset, training and predicting with new networks.

We'll start with some frequently useful imports...

In [5]:
import glob, json, os, sys
import datetime as dt
import numpy as np, pandas as pd, xarray as xr, matplotlib.pyplot as plt
from IPython.display import HTML

# We also set the logging level so that we get some feedback from the API
import logging
logging.basicConfig(level=logging.INFO)

## Download

The following is preparation of the downloaders, whose instantiation describes the interactions with the upstream APIs/data interfaces used to source various types of data. 

In [3]:
from icenet2.data.sic.mask import Masks
from icenet2.data.interfaces.cds import ERA5Downloader
from icenet2.data.sic.osisaf import SICDownloader

masks = Masks(north=False, south=True)
era5 = ERA5Downloader(
    var_names=["tas", "ta", "tos", "psl", "zg", "hus", "rlds", "rsds", "uas", "vas"],
    pressure_levels=[None, [500], None, None, [250, 500], [1000], None, None, None, None],
    dates=[pd.to_datetime(date).date() for date in
           pd.date_range("2019-12-28", "2020-12-31", freq="D")],
    delete_tempfiles=False,
    max_threads=64,
    north=False,
    south=True,
    # NOTE: there appears to be a bug with the toolbox API at present
    use_toolbox=False
)
sic = SICDownloader(
    dates=[pd.to_datetime(date).date() for date in
           pd.date_range("2019-12-28", "2020-12-31", freq="D")],
    delete_tempfiles=False,
    north=False,
    south=True,
)

INFO:root:Upping connection limit for max_threads > 10


Next we download all required data with our extended date range. All downloaders inherit a `download` method from the `Downloader` class in [`icenet2.data.producers`](https://github.com/JimCircadian/icenet2/blob/main/icenet2/data/producers.py), which also contains two other data producing classes `Generator` (which Masks inherits from) and `Processor` (used in the next section), each providing abstract implementations that multiple classes derive from.

In [4]:
# Original logging

In [4]:
# The original downloading takes a while, hence I left it over a weekend to preserve 
# the logging. When needing to pick up the processing, you can rerun these items
# to continue to regridding/rotating of fields

masks.generate(save_polarhole_masks=False)
era5.download()
sic.download()

INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_01.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200002021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_02.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200003021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_03.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200004021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_04.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200005021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_05.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200006021200.nc already exists
INFO:root:Saving ./data/masks/sh/masks/active_grid_cell_mask_06.npy
INFO:root:siconca ice_conc_sh_ease2-250_cdr-v2p0_200007021200.nc already exists
INFO:root:Saving ./data/masks/sh

The `ERA5Downloader` inherits from `ClimateDownloader`, from which several implementations derive their functionality. Two particularly useful methods shown below allow the downloaded data to be converted to the same grid and orientation as the OSISAF SIC data.

In [5]:
era5.regrid()
era5.rotate_wind_data()

INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_07_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_06_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_05_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_08_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_10_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_09_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_11_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_12_01.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_07_02.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_06_02.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_05_02.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_10_02.nc... 
INFO:root:Saving regridded data to ./data/era5/sh/psl/2020/2020_12_02.nc... 

It is hopefully obvious now that the CLI operations wrap several activities within the API up for convenience and initial ease of use, but that for experimentation, research and advancing the pipeline the API offers greater flexibility to manipulate processing chains as required for these purposes.

## Process

Similarly to the downloaders, each data producer (be it a `Downloader` or `Generator`) has a respective `Processor` that converts the `/data/` products into a normalised, preprocessed dataset under `/processed/` as per the `icenet_process_*` commands.

Firstly, to make life a bit easier, we set up some variables that are normally handled from the CLI arguments. In this case we're splitting the validation and test sets out of the 2020 data in a fairly naive manner.

In [9]:
processing_dates = dict(
    train=[pd.to_datetime(el) for el in 
           list(pd.date_range("2020-1-1", "2020-5-14")) + 
           list(pd.date_range("2020-6-1", "2020-8-14")) + 
           list(pd.date_range("2020-9-1", "2020-12-31"))],
    val=[pd.to_datetime(el) for el in pd.date_range("2020-5-15", "2020-5-31")],
    test=[pd.to_datetime(el) for el in pd.date_range("2020-8-15", "2020-8-31")],
)
processed_name = "notebook_api_data"

Next, we create the data producer and configure them for the dataset we want to create. 

In [10]:
from icenet2.data.processors.era5 import IceNetERA5PreProcessor
from icenet2.data.processors.meta import IceNetMetaPreProcessor
from icenet2.data.processors.osi import IceNetOSIPreProcessor

pp = IceNetERA5PreProcessor(
    ["uas", "vas"],
    ["tas", "ta500", "tos", "psl", "zg500", "zg250", "rsds", "rlds",
     "hus1000"],
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True
)
osi = IceNetOSIPreProcessor(
    ["siconca"],
    [],
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=["siconca"],
    linear_trend_days=93,
    north=False,
    south=True
)
meta = IceNetMetaPreProcessor(
    processed_name,
    north=False,
    south=True
)

2022-01-23 10:32:53.547239: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
INFO:root:Creating path: ./processed/notebook_api_data/era5
INFO:root:Creating path: ./processed/notebook_api_data/osisaf
INFO:root:Creating path: ./processed/notebook_api_data/meta


Next, we initialise the data processors using `init_source_data` which scans the data source directories to understand what data is available for processing based on the parameters. 

In [11]:
pp.init_source_data(
    lag_days=3,
)
pp.process()
osi.init_source_data(
    lag_days=3,
    lead_days=93,
)
osi.process()
meta.process()

INFO:root:Processing 332 dates for train category
INFO:root:Including lag of 3 days
INFO:root:Processing 17 dates for val category
INFO:root:Including lag of 3 days
INFO:root:Processing 17 dates for test category
INFO:root:Including lag of 3 days
INFO:root:Got 369 files for hus1000
INFO:root:Got 369 files for psl
INFO:root:Got 369 files for rlds
INFO:root:Got 369 files for rsds
INFO:root:Got 369 files for ta500
INFO:root:Got 369 files for tas
INFO:root:Got 369 files for tos
INFO:root:Got 369 files for uas
INFO:root:Got 369 files for vas
INFO:root:Got 369 files for zg250
INFO:root:Got 369 files for zg500
INFO:root:Opening files for uas
INFO:root:Filtered to 369 units long based on configuration requirements
INFO:root:Normalising uas
INFO:root:Opening files for vas
INFO:root:Filtered to 369 units long based on configuration requirements
INFO:root:Normalising vas
INFO:root:Opening files for tas
INFO:root:Filtered to 369 units long based on configuration requirements
INFO:root:Normalising 

At this point the preprocessed data is ready to convert or create a configuration for the network dataset.

### Dataset creation

As with the `icenet_dataset_create` command we can create a dataset configuration for training the network. As before this can include cached data for the network in the format of a TFRecordDataset compatible set of tfrecords. To achieve this we create the `IceNetDataLoader`, which can both generate `IceNetDataSet` configurations (which easily provide the necessary functionality for training and prediction) as well as individual data samples for direct usage.

In [13]:
from icenet2.data.loader import IceNetDataLoader

dl = IceNetDataLoader("loader.notebook_api_data.json",
                      "api_dataset",
                      3,
                      n_forecast_days=93,
                      north=False,
                      south=True,
                      output_batch_size=4,
                      generate_workers=32)

INFO:root:Creating path: ./network_datasets/api_dataset
INFO:root:Loading configuration loader.notebook_api_data.json


At this point we can either use `generate` or `write_dataset_config_only` to produce a ready-to-go `IceNetDataSet` configuration. 

In [14]:
dl.generate()

INFO:root:332 train dates in total, generating cache data.
INFO:root:83 tasks submitted
INFO:root:17 val dates in total, generating cache data.
INFO:root:88 tasks submitted
INFO:root:17 test dates in total, generating cache data.
INFO:root:93 tasks submitted
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000000.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000001.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000002.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000003.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000005.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000004.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000008.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/00000006.tfrecord
INFO:root:Finished output ./network_datasets/api_dataset/sh/train/000

## Train

For single runs we programmatically can call the same method used by the CLI. `train_model` defines the training process from start to finish. The [`model-ensembler`](https://github.com/JimCircadian/model-ensembler) works outside the API, controlling multiple CLI submissions. Customising an ensemble can be achieved through looking at the configuration in [the pipeline repository](https://github.com/antarctica/IceNet-Pipeline). That said, if workflow system integration (e.g. Airflow) is desired, integrating via this method is the way to go.

In [16]:
from icenet2.model.train import train_model
import tensorflow as tf

dataset_config = "dataset_config.api_dataset.json"
strategy = tf.distribute.get_strategy()

trained_path, history = \
    train_model("api_test_run",
                dataset_config,
                batch_size=4,
                epochs=10,
                n_filters_factor=0.8,
                seed=42,
                strategy=strategy,
                # == 2 for notebook usage
                training_verbosity=2,
                # Various other parameters can be set here as with the CLI
                # learning_rate=args.lr,
                # lr_10e_decay_fac=args.lr_10e_decay_fac,
                # lr_decay_start=args.lr_decay_start,
                # lr_decay_end=args.lr_decay_end,
                # pre_load_network=args.preload is not None,
                # pre_load_path=args.preload,
                # use_multiprocessing=args.multiprocessing,
                # use_wandb=not args.no_wandb,
                # wandb_offline=args.wandb_offline,
                # workers=args.workers, 
    )

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jambyr (use `wandb login --relogin` to force relogin)
/data/hpcdata/users/jambyr/miniconda3/envs/notebook-test/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


INFO:root:Hyperparameters: {'seed': 42, 'learning_rate': 0.0001, 'filter_size': 3, 'n_filters_factor': 0.8, 'lr_10e_decay_fac': 1.0, 'lr_decay': -0.0, 'lr_decay_start': 10, 'lr_decay_end': 30, 'batch_size': 4}
INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Datasets: 83 train, 5 val and 5 test filenames
2022-01-23 11:27:21.139001: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-23 11:27:21.141639: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-23 11:27:21.195506: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: Quadro P4000 computeCapability: 6.1
coreClock: 1.48GHz coreCount: 14 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 226.62GiB/s
2022-01-23 11:27:21.195579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dyn

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 432, 432, 13 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 432, 432, 51) 60639       input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 432, 432, 51) 23460       conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 432, 432, 51) 204         conv2d_1[0][0]                   
______________________________________________________________________________________________

2022-01-23 11:27:40.721989: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:177] Filling up shuffle buffer (this may take a while): 8 of 20
2022-01-23 11:27:45.608800: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:230] Shuffle buffer filled.
2022-01-23 11:27:46.772259: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-01-23 11:27:49.577483: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-01-23 11:27:51.765140: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.84GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-01-23 11:27:52.302383: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.84GiB with f

83/83 - 95s - loss: 257.3481 - mae: 42.4509 - rmse: 45.3512 - mse: 2056.7297 - val_loss: 208.6347 - val_mae: 36.4875 - val_rmse: 41.2718 - val_mse: 1701.2103

Epoch 00001: val_rmse improved from inf to 41.27184, saving model to ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5


...

INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 9/10
83/83 - 63s - loss: 15.8757 - mae: 21.0770 - rmse: 27.7343 - mse: 769.1904 - val_loss: 16.7101 - val_mae: 20.1856 - val_rmse: 25.6518 - val_mse: 656.4910

Epoch 00009: val_rmse did not improve from 24.60941


INFO:root:
Setting learning rate to: 9.999999747378752e-05



Epoch 10/10
83/83 - 63s - loss: 13.9138 - mae: 20.0423 - rmse: 26.8979 - mse: 723.4977 - val_loss: 15.1538 - val_mae: 20.7573 - val_rmse: 26.8361 - val_mse: 719.7869

Epoch 00010: val_rmse did not improve from 24.60941


INFO:root:Saving network to: ./results/networks/api_test_run/api_test_run.network_api_dataset.42.h5


Breaking `train_model` apart, one can look at customising the training process itself programmatically. Here, we've reduced `train_model` to its component parts with some notes about missing items (e.g. callbacks and wandb integration), to give some insight into how the training workflow is architected.

In [20]:
from icenet2.data.dataset import IceNetDataSet
from icenet2.model.models import unet_batchnorm
import icenet2.model.losses as losses
import icenet2.model.metrics as metrics

# train_model sets up wandb here
seed = 42
np.random.default_rng(seed)
tf.random.set_seed(seed)

ds = IceNetDataSet(dataset_config, batch_size=4)

input_shape = (*ds.shape, ds.num_channels)
train_ds, val_ds, test_ds = ds.get_split_datasets()

# train_model handles pickup runs/trained networks
run_name = "custom_run"
network_folder = os.path.join(".", "results", "networks", run_name)

if not os.path.exists(network_folder):
    logging.info("Creating network folder: {}".format(network_folder))
    os.makedirs(network_folder)

network_path = os.path.join(network_folder,
                            "{}.network_{}.{}.h5".format(run_name,
                                                         ds.identifier,
                                                         seed))

callbacks_list = list()
# train_model sets up various callbacks: early stopping, lr scheduler, 
# checkpointing, wandb and tensorboard

with strategy.scope():
    loss = losses.WeightedMSE()
    metrics_list = [
        metrics.WeightedMAE(),
        metrics.WeightedRMSE(),
        losses.WeightedMSE()
    ]

    network = unet_batchnorm(
        input_shape=input_shape,
        loss=loss,
        metrics=metrics_list,
        learning_rate=1e-4,
        filter_size=3,
        n_filters_factor=0.8,
        n_forecast_days=ds.n_forecast_days,
    )

# train_model loads weights
network.summary()

model_history = network.fit(
    train_ds,
    epochs=5,
    verbose=2,
    callbacks=callbacks_list,
    validation_data=val_ds,
    max_queue_size=10,
    workers=4,
    use_multiprocessing=True
)

logging.info("Saving network to: {}".format(network_path))
network.save_weights(network_path)


INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Datasets: 83 train, 5 val and 5 test filenames


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 432, 432, 13 0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 432, 432, 51) 60639       input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 432, 432, 51) 23460       conv2d_49[0][0]                  
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 432, 432, 51) 204         conv2d_50[0][0]                  
____________________________________________________________________________________________

INFO:root:Saving network to: ./results/networks/custom_run/custom_run.network_api_dataset.42.h5


As can be seen the training workflow is very standard for deep learning networks, with `train_model` and CLI wrapping up the training process with a lot of customisation of extraneous functionality. 

## Predict

In much the same manner as with `train_model`, the `predict_forecast` method acts as a convenient entry point workflow system integration, CLI entry as well as an overridable method upon which to base custom implementations. Using the method directly relies on loading from a prepared (but perhaps not cached) dataset.

Some parameters are fed to `predict_forecast` that ideally shouldn't need to be specified (like `seed` and `n_filters_factor`) and might seem contextually odd. They're used to locate the appropriate saved network. *This will be cleaned up in a future version*.  

In [6]:
from icenet2.model.predict import predict_forecast

# Same as our training set, we'll use the test dates defined when we created this
# dataset
dataset_config = "dataset_config.api_dataset.json"

# Follows the naming convention used by the CLI version
output_dir = os.path.join(".", "results", "predict",
                          "custom_run_forecast",
                          "{}.{}".format("custom_run", "42"))

forecasts, gen_outputs, sample_weights = \
    predict_forecast(dataset_config,
                     "custom_run",
                     n_filters_factor=0.8,
                     seed=42,
                     # Range previously defined as processing_dates["train"]
                     start_dates=[pd.to_datetime(el).date()
                                  for el in pd.date_range("2020-8-15", "2020-8-31")],
                     testset=True)

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Loading configuration loader.notebook_api_data.json
INFO:root:Datasets: 83 train, 5 val and 5 test filenames
2022-01-23 16:00:47.695406: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-23 16:00:47.696685: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3200000000 Hz
INFO:root:Processing batch 1 - item 0
INFO:root:Processing batch 1 - item 1
INFO:root:Processing batch 1 - item 2
INFO:root:Processing batch 1 - item 3
INFO:root:Processing batch 2 - item 0
INFO:root:Processing batch 2 - item 1
INFO:root:Processing batch 2 - item 2
INFO:root:Processing batch 2 - item 3
INFO:root:Processing batch 3 - item 0
INFO:root:Processing batch 3 - item 1
INFO:root:Processing batch 3 - item 2
INFO:root:Processing batch 3 - item 3
INFO:root:Processing batch 4 - item 0
INFO:root:Processing batch 4 - item 1
INFO:root:Pro

The persistence and respective use of these results is then up to the user, with the threefold outputs correlating to that which is normally saved to disk as individual files containing the numpy arrays by the CLI command.

The [internals of the `predict_forecast` method](https://github.com/JimCircadian/icenet2/blob/main/icenet2/model/predict.py#L17) are still undergoing some development, but it should be noted that this method can be easily overridden or called as part of a larger workflow. In particular, within this method it's worth noting the importance of the `testset` parameter. 

Should `testset` be true, then cached data generated in `network_datasets` is never used, and instead the preprocessed data in `processed` is used directly. This actually makes the implementation of `predict_forecast` extremely simple compared with the alternative, due to some outstanding work to derive dates from the cached batched files. 

As before this is revised implementation in order to illustrate the "non-testset" use case, so several modifications are in situ for notebook execution:

In [26]:
from icenet2.data.dataset import IceNetDataSet
from icenet2.model.models import unet_batchnorm
import tensorflow as tf

# Usually passed to predict forecast
start_dates = [pd.to_datetime(el).date()
               for el in pd.date_range("2020-8-15", "2020-8-31")]
testset = False
# End predict_forecast args

ds = IceNetDataSet(dataset_config)
dl = ds.get_data_loader()

if not testset:
    logging.info("Generating forecast inputs from processed/ files")

    forecast_inputs, gen_outputs, sample_weights = \
        list(zip(*[dl.generate_sample(date) for date in start_dates]))
else:
    # Use the network_dataset cached data, which is a much messier implementation
    # but worthwhile using if running massive datasets incl.datasets
    # ...
    pass

network_folder = os.path.join(".", "results", "networks", "custom_run")

dataset_name = ds.identifier
network_path = os.path.join(network_folder,
                            "{}.network_{}.{}.h5".format("custom_run",
                                                         "api_dataset",
                                                         42))

logging.info("Loading model from {}...".format(network_path))

network = unet_batchnorm(
    (*ds.shape, dl.num_channels),
    [],
    [],
    n_filters_factor=0.8,
    n_forecast_days=ds.n_forecast_days
)
network.load_weights(network_path)

predictions = []

for i, net_input in enumerate(forecast_inputs):
    logging.info("Running prediction {} - {}".format(i, start_dates[i]))
    pred = network(tf.convert_to_tensor([net_input]), training=False)
    predictions.append(pred)
print("Predictions: {} shape {}".format(len(predictions), 
                                        predictions[0].shape))
print("Generated outputs: {} shape {}".format(len(gen_outputs), 
                                              gen_outputs[0].shape))
print("Sample weights: {} shape {}".format(len(sample_weights), 
                                           sample_weights[0].shape))

INFO:root:Loading configuration dataset_config.api_dataset.json
INFO:root:Loading configuration loader.notebook_api_data.json
INFO:root:Generating forecast inputs from processed/ files
INFO:root:Loading model from ./results/networks/custom_run/custom_run.network_api_dataset.42.h5...
INFO:root:Running prediction 0 - 2020-08-15
INFO:root:Running prediction 1 - 2020-08-16
INFO:root:Running prediction 2 - 2020-08-17
INFO:root:Running prediction 3 - 2020-08-18
INFO:root:Running prediction 4 - 2020-08-19
INFO:root:Running prediction 5 - 2020-08-20
INFO:root:Running prediction 6 - 2020-08-21
INFO:root:Running prediction 7 - 2020-08-22
INFO:root:Running prediction 8 - 2020-08-23
INFO:root:Running prediction 9 - 2020-08-24
INFO:root:Running prediction 10 - 2020-08-25
INFO:root:Running prediction 11 - 2020-08-26
INFO:root:Running prediction 12 - 2020-08-27
INFO:root:Running prediction 13 - 2020-08-28
INFO:root:Running prediction 14 - 2020-08-29
INFO:root:Running prediction 15 - 2020-08-30
INFO:r

Predictions: 17 shape (1, 432, 432, 93)
Generated outputs: 17 shape (432, 432, 93, 1)
Sample weights: 17 shape (432, 432, 93, 1)


## Summary

This notebook has attempted to illustrate the workflow implementations of the CLI as well as highlight the flexibility of direct integration using it. Ultimately, library usage is the only way to achieve truly novel and flexible usage, the CLI is for convenience of running existing pipelines without having to manually implement complex scripts. 

The key to leveraging the benefits of both of these interfaces being provided is to consider using the following workflow:

* Get your environment(s) set up, be they research, development or production
* Use the existing CLI implementations to seed the data stores and get baseline networks operational
* Start to customise the existing operations via custom calls to the API, for example by downloading new variables or adding extra analysis to training/prediction runs
* If researching, consider [extending the functionality of the API to include revised or completely new implementations, such as additional data sources](04.library_extension.ipynb)

This last point brings us to the topic of the last of the introductory notebooks. 

## Version
- Codebase: v0.1.0